In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

In [4]:
%matplotlib inline

In [5]:
TEXT_COLOUR = {
    'PURPLE':'\033[95m',
    'CYAN':'\033[96m',
    'DARKCYAN':'\033[36m',
    'BLUE':'\033[94m',
    'GREEN':'\033[92m',
    'YELLOW':'\033[93m',
    'RED':'\033[91m',
    'BOLD':'\033[1m',
    'UNDERLINE':'\033[4m',
    'END':'\033[0m'
}

def print_bold(*msgs):
    print(TEXT_COLOUR['BOLD'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_green(*msgs):
    print(TEXT_COLOUR['GREEN'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_error(*msgs):
    print(TEXT_COLOUR['RED'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def wrap_green(msg):
    return TEXT_COLOUR['GREEN'] + msg + TEXT_COLOUR['END']

def wrap_red(msg):
    return TEXT_COLOUR['RED'] + msg + TEXT_COLOUR['END']

def up_down_str(val):
    msg = str(val)
    if val > 0:
        msg = wrap_green(msg)
    elif val < 0:
        msg = wrap_red(msg)
    return msg

In [6]:
exp='bert-large'
num_layers = 24

In [7]:
tasks = ["CoLA","SST-2","MRPC","STS-B","QQP","MNLI", "MNLI-MM", "QNLI", "RTE"]
tasks = ["CoLA","SST-2","MRPC","STS-B","RTE"]

metrics = {
    "CoLA":["mcc"],
    "MNLI":["acc"],
    "MNLI-MM":["acc"],
    "MRPC":["f1"],
    "QNLI":["acc"],
    "QQP":["f1"],
    "RTE":["acc"],
    "SST-2":["acc"],
    "STS-B":["spearmanr"],
    "WNLI":["acc"] #temp
}

reported_in_paper = {
    "CoLA":0.00,
    "MNLI":0.00,
    "MNLI-MM":0.0,
    "MRPC":0.00,
    "QNLI":0.00,
    "QQP":0.00,
    "RTE":0.00,
    "SST-2":0.00,
    "STS-B":0.00,
    "WNLI":0.00
}

In [8]:

def get_average_val(lines):
    reported = []
    for line in lines:
#         print('\t', line)
        val = float(line.split('\t')[1])
        if val != 0:
            reported.append(val)
    out = 0
    if len(reported) != 0:
        reported.sort(reverse = True)
        candidates = [reported[0]]
        for j in range(1, len(reported)):
            if reported[j] > 0.9 * reported[0]:
                candidates.append(reported[j])
        out = np.mean(candidates)
        
    return out


In [9]:
results = {}

for task in tasks:
    task_results = {}
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        # base metrics
        print(f"../../mt_dnn_exp_results/{exp}/{task}/base-{metric}.txt")
        f=open(f"../../mt_dnn_exp_results/{exp}/{task}/base-{metric}.txt", "r")
        lines = f.read().splitlines()
        task_results[f'base-{metric}'] = get_average_val(lines)
        
        # no layer metrics
        
        fine_tuning_metrics = []
        f=open(f"../../mt_dnn_exp_results/{exp}/{task}/no_layer-{metric}.txt", "r")

        lines = f.read().splitlines()
        fine_tuning_metrics.append(get_average_val(lines))
        
        # fine-tuned metrics
        
        log_file_prefix=''
        for i in reversed(range(int(num_layers/2), num_layers)):
            log_file_prefix += str(i)
            f=open(f"../../mt_dnn_exp_results/{exp}/{task}/{log_file_prefix}-{metric}.txt", "r")
            lines = f.read().splitlines()
            fine_tuning_metrics.append(get_average_val(lines))
            
            log_file_prefix +='_'
        
        task_results[f'{metric}'] = list(reversed(fine_tuning_metrics))
        
    results[task] = task_results

../../mt_dnn_exp_results/bert-large/CoLA/base-mcc.txt
../../mt_dnn_exp_results/bert-large/SST-2/base-acc.txt
../../mt_dnn_exp_results/bert-large/MRPC/base-f1.txt
../../mt_dnn_exp_results/bert-large/STS-B/base-spearmanr.txt
../../mt_dnn_exp_results/bert-large/RTE/base-acc.txt


In [10]:
x_axis = []

for i in range(int(num_layers/2), num_layers):
    x_axis.append(str(i))

x_axis.append("none")

In [11]:
def draw_graph(task, y_label, paper, base, reported):
    plt.figure(figsize=(10,6))
    plt.plot(x_axis, reported)
    
    plt.xlabel("layers")
    plt.ylabel(y_label)
    
    if paper == 0.0:    
        gap = max(reported) - min(reported)
        top = max(max(reported), base) + (gap*0.2)
        bottom = min(min(reported), base) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
    else:
        gap = max(reported) - min(reported)
        top = max(max(reported), base, paper) + (gap*0.2)
        bottom = min(min(reported), base, paper) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
        plt.axhline(y=paper, linestyle='--', c='red')
    
    plt.title(f'{exp}-{task} ({round(base,4)})')
    plt.savefig(f'images/{exp}/{task}', format='png', bbox_inches='tight')
    plt.show()

In [12]:
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        reported = task_results[metric]
        base = task_results[f'base-{metric}']
        print_bold(task, metric, ': b -', round(base * 100, 2), 'h -',round(task_results[metric][0] * 100, 2), 'n -', round(task_results[metric][-1] * 100, 2))


CoLA mcc : b - 59.55 h - 58.17 n - 57.82


SST-2 acc : b - 92.87 h - 92.68 n - 93.1


MRPC f1 : b - 91.08 h - 90.52 n - 87.73


STS-B spearmanr : b - 91.11 h - 90.65 n - 88.52


RTE acc : b - 80.43 h - 81.81 n - 70.18



In [13]:
import copy 

layer_90 = []
layer_95 = []

threshold_90 = 0.9
threshold_95 = 0.95
x_axis.reverse()

for task in tasks:
#     print_bold(task)
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        flag_90 = True
        flag_95 = True

        for ind, val in enumerate(reported):

            if val/base > threshold_90 and flag_90:
                flag_90 = False
                layer_90.append(ind)
                results[task]['90%'] = ind

            if val/base > threshold_95 and flag_95:
                flag_95 = False
                layer_95.append(ind)
                results[task]['95%'] = ind

        if flag_90:
            print(task, "Fails to achieve 90% threshold", reported[-1]/base)
            layer_90.append(len(reported)-1)
            results[task]['90%'] = "-"

        if flag_95:
            print(task, "Fails to achieve 95% threshold", reported[-1]/base)
            layer_95.append(len(reported)-1)
            results[task]['95%'] = "-"


            
print(x_axis)
            
            
print(layer_90)
min_layer_ind_90 = max(layer_90)
print("layer_90 ", min_layer_ind_90, 'layer:', x_axis[min_layer_ind_90], round((1-(min_layer_ind_90/num_layers)) * 100, 2), '%')

print(layer_95)
min_layer_ind_95 = max(layer_95)
print("layer_95 ", min_layer_ind_95, 'layer:', x_axis[min_layer_ind_95], round((1-(min_layer_ind_95/num_layers)) * 100, 2), '%')


firsts = []
seconds = []
    
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        if task != "CoLA":
            first = round(100*reported[0]/base, 2)
            second = round(100*reported[1]/base, 2)
            firsts.append(first)
            seconds.append(second)
            
        print_bold(task, base)
        print('\t90', reported[min_layer_ind_90], round(reported[min_layer_ind_90]/base * 100, 2))
        print('\t95', reported[min_layer_ind_95], round(reported[min_layer_ind_95]/base * 100, 2))
        
print_bold(len(firsts), np.mean(firsts), np.mean(seconds), round(np.mean(seconds) - np.mean(firsts),2))

['none', '23', '22', '21', '20', '19', '18', '17', '16', '15', '14', '13', '12']
[0, 0, 0, 0, 3]
layer_90  3 layer: 21 87.5 %
[0, 0, 0, 0, 5]
layer_95  5 layer: 19 79.17 %

CoLA 0.5954824033575099

	90 0.5750243606459752 96.56
	95 0.5796888623577029 97.35

SST-2 0.9286697247706422

	90 0.9307339449541285 100.22
	95 0.9291284403669724 100.05

MRPC 0.9108452337378727

	90 0.8849981069764002 97.16
	95 0.8949508872917088 98.25

STS-B 0.9110897685123106

	90 0.8919742700870092 97.9
	95 0.8957122564903071 98.31

RTE 0.804332129963899

	90 0.7342960288808664 91.29
	95 0.7826714801444042 97.31

4 95.245 95.13000000000001 -0.11



In [14]:
for task in ["STS-B"]:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        print(task_results[metric][-1])
        print(task_results[metric][-2])

0.8852243389415211
0.8891767744058392


In [15]:
latex_metrics = {
    "CoLA":"MCC",
    "MNLI":"Acc.",
    "MNLI-MM":"Acc.",
    "MRPC":"F$_1$",
    "QNLI":"Acc.",
    "QQP":"F$_1$",
    "RTE":"Acc.",
    "SST-2":"Acc.",
    "STS-B":"$\\rho$"
}

In [16]:
print("\\begin{center}\n\t\\scalebox{0.7}{\n\t\t\\begin{tabular}{rc|ccccccccccccc} \n\t\t\\toprule[1pt] \n\t\t\\multirow{2}{*}{Task (metric)} & \\multirow{2}{*}{Baseline} & \\multicolumn{13}{c}{Fine-tuned layers} \\\\ \n\t\t\\cline{3-15} \n\t\t& & 12-23 & 13-23 & 14-23 & 15-23 & 16-23 & 17-23 & 18-23 & 19-23 & 20-23 & 21-23 & 22-23 & 23-23 & None \\\\ \n\t\t\t\\midrule")

avg_performance = []

for task in tasks:
    m = metrics[task][0]
    base_key = f"base-{m}"
    
    if task == "MNLI-MM":
        row = f"\t\t\tMNLI-mm ({latex_metrics[task]}) & "
    else:
        row = f"\t\t\t{task} ({latex_metrics[task]}) & "
    
    row += "{:0.2f}".format(round(results[task][base_key] * 100, 2))
    
    for ind, val in enumerate(results[task][m]):
        row += " & {:0.2f}".format(round(val * 100,2))
        
        if len(avg_performance) == ind:
            avg_performance.append([])
            
            
        percent = (val / results[task][base_key]) * 100
        avg_performance[ind].append(percent)
        
#     row += "& {}".format(results[task]["90%"])
#     row += "& {}".format(results[task]["95%"])
        
    row += " \\\\"
    print(row)
    
print("\t\t\t\\midrule\\midrule")

row = "\t\t\tRel. perf. (\%) & 100.00"

for perf in avg_performance:
    row += " & {:0.2f}".format(round(np.mean(perf) ,2))
    
row += " \\\\"

print(row)
    
print("\t\t\\end{tabular}\n\t}\n\t\\caption{MTDNN-BERT-large on GLUE}\n\t\\label{table:finetune-all}\n\\end{center}")

\begin{center}
	\scalebox{0.88}{
		\begin{tabular}{rc|ccccccccccccc} 
		\toprule[1pt] 
		\multirow{2}{*}{Task (Metric)} & \multirow{2}{*}{Baseline} & \multicolumn{13}{c}{Fine-tuned layers} \\ 
		\cline{3-18} 
		& & 12-23 & 13-23 & 14-23 & 15-23 & 16-23 & 17-23 & 18-23 & 19-23 & 20-23 & 21-23 & 22-23 & 23-23 & None \\ 
			\midrule
			CoLA (MCC) & 59.55 & 58.17 & 57.82 & 57.97 & 57.21 & 56.90 & 57.60 & 57.71 & 57.97 & 58.03 & 57.50 & 57.51 & 57.32 & 57.82 \\
			SST-2 (Acc.) & 92.87 & 92.68 & 92.91 & 92.78 & 92.52 & 92.57 & 92.91 & 92.84 & 92.91 & 92.71 & 93.07 & 93.14 & 93.10 & 93.10 \\
			MRPC (F$_1$) & 91.08 & 90.52 & 90.62 & 90.41 & 90.18 & 90.35 & 90.05 & 89.91 & 89.50 & 88.86 & 88.50 & 88.19 & 87.65 & 87.73 \\
			STS-B ($\rho$) & 91.11 & 90.65 & 90.47 & 90.31 & 90.20 & 90.11 & 90.00 & 89.84 & 89.57 & 89.37 & 89.20 & 89.11 & 88.92 & 88.52 \\
			RTE (Acc.) & 80.43 & 81.81 & 81.08 & 81.30 & 81.16 & 80.87 & 80.07 & 79.78 & 78.27 & 74.01 & 73.43 & 71.05 & 69.53 & 70.18 \\
			\midrule\mid